<a href="https://colab.research.google.com/github/thomaslu678/gee-test/blob/main/clean/9_import_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOTE: Requires MLP.pt file

In [86]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from datetime import timedelta
import scipy.stats as stats
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize
import geopandas as gpd
from shapely.geometry import Point
import requests

In [87]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import shap

In [88]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
class MLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),

            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),

            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

In [93]:
model = MLP(input_dim=10)

In [94]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Year 2/Fall 2025/HONOR 3700/Data/2/MLP.pt'))

<All keys matched successfully>

In [95]:
model.eval()

MLP(
  (net): Sequential(
    (0): Linear(in_features=10, out_features=64, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [111]:
new_scaler = StandardScaler()

In [114]:
old_mean_ = [0.13840289361946037,
 -0.0035840483502844306,
 0.0035840483502844306,
 0.10763790818960244,
 39.92871816262701,
 127.01074195536044,
 37.569997248409514,
 169.7159980720383,
 1.6843630583677387,
 6.302724969751013]
old_scale_ = [1.8904474651034928,
 0.7058617778121368,
 0.7058617778121368,
 0.07238789915216501,
 11.14120826066698,
 0.01987824586858675,
 0.002142538628953896,
 94.80918981636496,
 1.7983734482846312,
 3.3555003650064874]

In [121]:
new_scaler.mean_ = np.array(old_mean_)
new_scaler.scale_ = np.array(old_scale_)

In [123]:
test_input = np.array([0.008107, -0.042450, 0.042450, 0.185196, 18.979367, 317.553292, 1, 12, -0.029627, 3.3725176e+00])

single_input_scaled = new_scaler.transform(test_input.reshape(1, -1))  # Shape it to 2D for scaler
single_input_tensor = torch.tensor(single_input_scaled, dtype=torch.float32)

model.eval()
with torch.no_grad():
    predicted_label = model(single_input_tensor).cpu().numpy()  # Convert the output to a NumPy array

print(f"Predicted label: {predicted_label}")

Predicted label: [[338.3184]]
